<img style="float: right;" src="https://dm2302files.storage.live.com/y4pX8blza_49egU5TSGAB_Dj2UJtB-3-FVoNFY285_54WAl1Rcc43ZzIiQQKfuAXOwZs2g9L2_9Isy_B-WGuNOIQ5AMma7VrGsfiHQiXwdBjxSPWE-EZ53B1RqPi19j66aPPWLk8DjN2Yuqc3LCV_Q8t6-NT3jYWdNGJtSKytvN6i3sEc7J8qNlY28XTYuKH0m6G6n0dBYMZTqcHPVX39V-Rkf-o5D27FBKKi4zc_6deV0/WhatsApp%20Image%202020-05-26%20at%2015.14.51.jpeg?psid=1&width=640&height=97" width="300">

# The Battle of Neighborhoods

<span style="color:gray">*IBM Applied Data Science Capstone - Week 4*</span>

## 1. Introduction

### 1.1 Background

Evaluating houses is a deeply personal and complex process, impacted by diverse factors ranging from the physical characteristics and local amenities to politic-economic factors. It's important to analyze the situation, research the options, and gather all the necessary information before making an important decision about moving.

Besides the intrinsic features of a property such as the number of beds, baths, square footage, price, age, etc, a  factor that can affect the decision of evaluating houses is the proximity to the things that matter most to them. This may include a workplace, views, parks, schools, community service, residences of relatives and so on.

The same rings true for all who seeks a new place around the world in pursuit of their dreams or in search of a better life.

### 1.2 Problem

Moving for work is a different than simply moving, typically because the timeline involved in taking a job in a new location is a lot shorter than when you decide on a change of scenery and then focus on getting the new position.

One of my clients lives in Marble Hill, Manhattan, New York and she loves her neighborhood. She recieved a great job offer from Toronto, and she decided to move to Toronto in 3 weeks to take up the new opportunity. She wants to find out a neighborhood in Toronto that has similar amenities available near her that she gets in Marble Hill. 

In this project Python's data analysis and geospatial analysis packages was used to analyze the whole spectrum of available listings in a market, evaluate and score properties based on various attribute and spatial parameters and arrive at a shortlist of neighborhoods similars to Marble Hill.

### 1.3 Target Audience

Target audience for this project is anyone who is searching for a new properties in neighborhoods that provides similar amenities of their current neighborhood.

## 2. Data collection and cleaning

### 2.1 Data sources

Housing data for the city of New York was collected from the New York University Libraries <span style="color:blue">[1]</span>; and the data for the city of Toronto was scraped from Wikipedia <span style="color:blue">[2]</span>. Data was read using Pandas as DataFrame objects. These DataFrames form the foundation of this study upon which both spatial and attribute analysis are performed.

[1] https://geo.nyu.edu/catalog/nyu_2451_34572

[2] https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Foursquare API was used to explore Manhattan, New York City and neighborhoods in Toronto. The explore function was used to get the most common venue categories in each neighborhood, and then the neighborhoods was grouped into clusters. To complete this task k-means clustering algorithm was used. Finally, the Folium library was used to visualize the neighborhoods in New Toronto and their neighborhoods similars to Bronx.

#### Import required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from tqdm.notebook import tqdm # progress bar library

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import warnings # warnings library
warnings.filterwarnings("ignore")

print('Libraries imported.')

Libraries imported.


#### Create New York DataFrame

In [2]:
# Get New York Dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

newyork_data = newyork_data['features']

# Define the DataFrame columns
columns_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# Instantiate the dataframe
ny_df = pd.DataFrame(columns=columns_names)

for data in newyork_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_df = ny_df.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
ny_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Create Toronto DataFrame

In [3]:
# Get tables from the URL and transforme into a DataFrame
tr_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0] # index 0 is the table of interest

# Rename columns
columns_names = ["PostalCode", "Borough", "Neighborhood"]
tr_df.columns = columns_names

# Drop cells with a borough that is "Not assigned"
tr_df = tr_df[tr_df.Borough != "Not assigned"].reset_index(drop=True)

# Make neighborhood equals the borough if neighborhood is "Not assigned"
for index, row in tr_df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

# Convert Neighborhood strings to list
tr_df['Neighborhood'] = tr_df.Neighborhood.str.split(',', expand=False)

# Explode Neighborhood list to single rows
tr_df = tr_df.explode('Neighborhood').reset_index(drop=True)

tr_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor


In [4]:
# Define a function to get coordinates
def get_latlng(neighborhood):
    # Initialize your variable to None
    lat_lng_coords = None
    # Loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Canada'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [5]:
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [get_latlng(neighborhood) for neighborhood in tqdm(tr_df["Neighborhood"].tolist(), 'Getting latitudes and longitudes')]

# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# Merge the coordinates into the original dataframe
tr_df['Latitude'] = df_coords['Latitude']
tr_df['Longitude'] = df_coords['Longitude']

# Check the neighborhoods and the coordinates
print(tr_df.shape)

tr_df.head()


(217, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.686575,-79.409993
1,M4A,North York,Victoria Village,43.731540,-79.314280
2,M5A,Downtown Toronto,Regent Park,43.660690,-79.360310
3,M5A,Downtown Toronto,Harbourfront,43.639510,-79.383160
4,M6A,North York,Lawrence Manor,43.722940,-79.431160


---

<h4>Author:  <a href="https://br.linkedin.com/in/henrique-mand">Henrique Mandt</a></h4>
<p><a href="https://br.linkedin.com/in/henrique-mand">Henrique Mandt</a>, Civil Engineer and Consultant with a track record of developing soluctions that substantially increases operational efficiency, mitigate risks and maximize benefits for teams, investors and customers. He is a Data Scientist enthusiast with interest in data mining, machine learning and spatial statistical modelling.</p>

---